In [4]:
# Constants for Salsa20 with different key lengths
C128 = [0x61707865, 0x3120646e, 0x79622d36, 0x6b206574]  # "expand 16-byte k" - 128bit
C256 = [0x61707865, 0x3320646e, 0x79622d32, 0x6b206574]  # "expand 32-byte k" -256bit
C64 = [0x61707865, 0x3420646e, 0x79622d31, 0x6b206574]  # "expand 8-byte k" - 64bit key

# rotation function
def rotl32(x, n):
#Rotate left for 32-bit integers
    return ((x << n) & 0xFFFFFFFF) | (x >> (32 - n))

def quarterround(y0, y1, y2, y3):
#Salsa20 quarterround
    z1 = y1 ^ rotl32((y0 + y3) & 0xFFFFFFFF, 7)
    z2 = y2 ^ rotl32((z1 + y0) & 0xFFFFFFFF, 9)
    z3 = y3 ^ rotl32((z2 + z1) & 0xFFFFFFFF, 13)
    z0 = y0 ^ rotl32((z3 + z2) & 0xFFFFFFFF, 18)
    return z0, z1, z2, z3

def rowround(y):
#Salsa20 rowround
    z = list(y)
    z[0], z[1], z[2], z[3] = quarterround(y[0], y[1], y[2], y[3])
    z[5], z[6], z[7], z[4] = quarterround(y[5], y[6], y[7], y[4])
    z[10], z[11], z[8], z[9] = quarterround(y[10], y[11], y[8], y[9])
    z[15], z[12], z[13], z[14] = quarterround(y[15], y[12], y[13], y[14])
    return z

def columnround(y):
#Salsa20 columnround
    z = list(y)
    z[0], z[4], z[8], z[12] = quarterround(y[0], y[4], y[8], y[12])
    z[5], z[9], z[13], z[1] = quarterround(y[5], y[9], y[13], y[1])
    z[10], z[14], z[2], z[6] = quarterround(y[10], y[14], y[2], y[6])
    z[15], z[3], z[7], z[11] = quarterround(y[15], y[3], y[7], y[11])
    return z

def doubleround(y):
#Salsa20 doubleround
    return rowround(columnround(y))

#Salsa20 hash function
def salsa20_hash(x):
    z = list(x)
    for i in range(4):  # Salsa20/8, 8 rounds, 4 doublerounds
        z = doubleround(z)
    return [(z[i] + x[i]) & 0xFFFFFFFF for i in range(16)]

#cite, pls ignore
# def bytes_to_words(b):
#     """Convert a byte array to a list of 32-bit words (little-endian)
#     return [int.from_bytes(b[i:i+4], 'little') for i in range(0, len(b), 4)]
def bytes_to_words(b):
#Convert a byte array to a list of 32-bit words (little-endian)
    return [(b[i] | (b[i+1] << 8) | (b[i+2] << 16) | (b[i+3] << 24)) for i in range(0, len(b), 4)]

#cite, pls ignore
# def words_to_bytes(w):
#     """Convert a list of 32-bit words to a byte array (little-endian)
#     return b''.join(word.to_bytes(4, 'little') for word in w)
def words_to_bytes(w):
#Convert a list of 32-bit words to a byte array (little-endian)
    return bytes([(w[i] >> (8 * j)) & 0xFF for i in range(len(w)) for j in range(4)])

def salsa20_expand(key, nonce, counter):
#Expand key and nonce into a Salsa20 keystream block
    if len(key) == 8:  # 64-bit key, 8 bytes
        # C64
        k = bytes_to_words(key)  # 2 words for 64-bit key
    elif len(key) == 16:  # 128-bit key, 16 bytes
        # C128
        k = bytes_to_words(key)  # 4 words for 128-bit key
    elif len(key) == 32:  # 256-bit key, 32 bytes
        # C256
        k = bytes_to_words(key)  # 8 words for 256-bit key
    else:
        raise ValueError("Key length must be 8, 16, or 32 bytes.")

    n_words = bytes_to_words(nonce)

    # build the Salsa20 state based on the key length
    if len(key) == 8:
        state = [
            C64[0], k[0], k[1], k[0],
            k[1], C64[1], n_words[0], n_words[1],
            counter, 0, C64[2], k[0],
            k[1], k[0], k[1], C64[3]
        ]
    elif len(key) == 16:
        state = [
            C128[0], k[0], k[1], k[2],
            k[3], C128[1], n_words[0], n_words[1],
            counter, 0, C128[2], k[0],
            k[1], k[2], k[3], C128[3]
        ]
    else:
        state = [
            C256[0], k[0], k[1], k[2],
            k[3], C256[1], n_words[0], n_words[1],
            counter, 0, C256[2], k[4],
            k[5], k[6], k[7], C256[3]
        ]
    return salsa20_hash(state)

def salsa20_encrypt(key, nonce, plaintext):
#Encrypt or decrypt using Salsa20
    ciphertext = bytearray()
    counter = 0  #counter init as 0 for the first block
    for i in range(0, len(plaintext), 64):
        keystream_block = salsa20_expand(key, nonce, counter)
        keystream = words_to_bytes(keystream_block)
        block = plaintext[i:i + 64]
        ciphertext_block = bytes([b ^ k for b, k in zip(block, keystream)])  # XOR with keystream.
        ciphertext.extend(ciphertext_block)
        counter += 1  # increment counter after processing a block.
    return ciphertext

def is_hex_string(s):
#Check if a string is a valid hexadecimal string
    try:
        int(s, 16)
        return True
    except ValueError:
        return False

def input_key_and_nonce(key_length):
#Input key and nonce
    while True:
        key = input(f"Please input key ({key_length}-bit hex string): ").strip()
        if is_hex_string(key) and len(key) == key_length // 4:
            break
        print(f"Invalid key, Please enter a valid {key_length}-bit hexadecimal string.")
    key = bytes.fromhex(key)
    while True:
        nonce = input("Please input nonce (64-bit hex string): ").strip()
        if is_hex_string(nonce) and len(nonce) == 16:
            break
        print("Invalid nonce, Please enter a valid 64-bit hexadecimal string.")
    nonce = bytes.fromhex(nonce)
    return key, nonce

def main():
#Main function
    print("Salsa20/8 Stream Cipher, Please input one by one as guide")
    key_length = int(input("Please input key length in bits (non-standard-64, 128, or 256): ").strip())
    if key_length not in [64, 128, 256]:
        raise ValueError("Key length must be 64, 128, or 256 bits.")
    key, nonce = input_key_and_nonce(key_length)
    text = input("Type in text string (hexadecimal format, C/P): ").strip()
    if not is_hex_string(text):
        raise ValueError("Invalid Hexadecimal String")
    text_bytes = bytes.fromhex(text)
    result = salsa20_encrypt(key, nonce, text_bytes)
    result_hex = result.hex()
    print("Result (hex):", result_hex)
    # try to convert result to a readable string text, cipher keeps text
    try:
        result_text = result.decode('utf-8')
        print("Result (text):", result_text)
    except UnicodeDecodeError:
        print("For non-printable characters, show as hex.")
        print("Result (text):", result_hex)

if __name__ == "__main__":
    main()


Salsa20/8 Stream Cipher, Please input one by one as guide


KeyboardInterrupt: Interrupted by user